In [8]:
# import easygui
# print( easygui.fileopenbox() )

/media/alexn/Data_2/Neuroglancer_Files/Neuroglancer_Volumes/Matrix/MD585/human_annotation/combined_volume/wholebrain_xy5um_z20um/color_1_thickness_x_offset_0/volume_colored.npy


In [1]:
import numpy as np
import pickle as pkl
import boto3
import sys
import os

import json
import pathlib
from neuroglancer_scripts.scripts import (generate_scales_info,
                                          slices_to_precomputed,
                                          compute_scales, volume_to_precomputed)
import nibabel as nib
import re


sys.path.append(os.path.join( os.environ['NG_REPO_DIR'], 'src', 'utilities'))
from filepath_manager import *

In [27]:
# s3_creds_file = '/mnt/c/Users/Alex/Documents/json_credentials/alex_aws_credentials.json'
# bucket_name = 'test-bucket-sid'

In [2]:
stack = 'MD585'
xy_res=5
z_res=20

source_dir = get_volume_fp( stack, precomputed=False, human_annotated=True, volume_type='combined', brain_crop='brainstem', \
                xy_res=xy_res, z_res=z_res, offset=False, color_scheme=1, thickness_scheme=1, structure=None ) + \
                'volume_colored.npy'

out_dir = get_volume_fp( stack, precomputed=True, human_annotated=True, volume_type='combined', brain_crop='brainstem', \
                  xy_res=xy_res, z_res=z_res, offset=False, color_scheme=1, thickness_scheme=1, structure=None )
ni_out = out_dir + 'vol_color.nii'

'/media/alexn/Data_2/Neuroglancer_Files/Neuroglancer_Volumes/Matrix/MD585/human_annotation/combined_volume/brainstem_xy5um_z20um/color_1_thickness_1_offset_0/'

In [30]:
vol_m = np.load(source_dir)

! mkdir $out_dir

vol_m = np.swapaxes(vol_m,0,2)
print(vol_m.shape)

z_res_mm = z_res*0.001
xy_res_mm = xy_res*0.001
vol_img = nib.Nifti1Image(vol_m, affine=np.array(\
      [[ xy_res_mm, 0.,         0.,        0.],\
       [ 0.,        xy_res_mm,  0.,        0.],\
       [ 0.,        0.,         z_res_mm,  0.],\
       [ 0.,        0.,         0.,        1.]]))
nib.save(vol_img, ni_out)

volume_to_precomputed.main(['', ni_out, out_dir, '--generate-info', '--no-gzip'])

In [33]:
with open(os.path.join(os.path.join(out_dir, 'info_fullres.json')), 'r') as info_file:
    info = json.load(info_file)

info["type"] = "segmentation"

with open(os.path.join(os.path.join(out_dir, 'info_fullres.json')), 'w') as info_file:
    json.dump(info, info_file)

In [34]:
generate_scales_info.main(['', os.path.join(out_dir, 'info_fullres.json'),  '--encoding' , 'compressed_segmentation',
                           out_dir, '--max-scales', '1', '--target-chunk-size', '128'])

volume_to_precomputed.main(['', ni_out, out_dir, '--flat', '--no-gzip'])

0


writing: 100%|██████████| 150/150 [00:57<00:00, 11.28chunks/s]

In [40]:
# out_dir = '/mnt/x/temp_s3_download/volumes/precomputed/'+stack+'/'
! aws s3 cp --recursive $out_dir s3://test-bucket-sid/final_precomputed_volumes/$stack/

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_256-288 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_256-288
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_192-256 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_192-256
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/0-256_1024-1099_64-128 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/0-256_1024-1099_64-128
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD58

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_256-512_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_256-512_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_512-768_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_512-768_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_512-768_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_512-768_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_512-768_256-288 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_512-768_256-288
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/256-512_256-512_64-128 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/256-512_256-512_64-128
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD58

upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_1024-1099_256-288 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_1024-1099_256-288
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_1024-1099_64-128 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_1024-1099_64-128
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_0-64 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_0-64
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_128-192 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_128-192
upload: ../../../../../../mnt/x/temp_s3_download/volumes/precomputed/MD589/5um/768-1024_256-512_192-256 to s3://test-bucket-sid/final_precomputed_volumes/MD589/5um/768-1024_256-512_192-256
upload: ../../../../../../mnt/x/temp_s3_download/volume